Python code for cryptanalysis of a monoalphabetic substitution cipher using frequency analysis:

In [ ]:
import string

# define expected letter frequencies for English language
expected_frequencies = {
    'a': 0.08167, 'b': 0.01492, 'c': 0.02782, 'd': 0.04253,
    'e': 0.12702, 'f': 0.02228, 'g': 0.02015, 'h': 0.06094,
    'i': 0.06966, 'j': 0.00153, 'k': 0.00772, 'l': 0.04025,
    'm': 0.02406, 'n': 0.06749, 'o': 0.07507, 'p': 0.01929,
    'q': 0.00095, 'r': 0.05987, 's': 0.06327, 't': 0.09056,
    'u': 0.02758, 'v': 0.00978, 'w': 0.0236, 'x': 0.0015,
    'y': 0.01974, 'z': 0.00074
}

def frequency_analysis(ciphertext):
    """
    Perform frequency analysis on a ciphertext and return a substitution key
    """
    # count the frequency of each letter in the ciphertext
    freq = {}
    for letter in ciphertext:
        if letter.isalpha():
            if letter.lower() not in freq:
                freq[letter.lower()] = 1
            else:
                freq[letter.lower()] += 1
                
    # calculate the frequency distribution
    total = sum(freq.values())
    distribution = {}
    for letter in freq:
        distribution[letter] = freq[letter] / total
    
    # sort the letters by frequency
    sorted_letters = sorted(distribution, key=distribution.get, reverse=True)
    
    # create the substitution key
    key = {}
    for i in range(len(sorted_letters)):
        key[sorted_letters[i]] = list(expected_frequencies.keys())[i]
    
    return key

def decrypt(ciphertext, key):
    """
    Decrypt a ciphertext using a substitution key
    """
    plaintext = ''
    for letter in ciphertext:
        if letter.isalpha():
            if letter.isupper():
                plaintext += key[letter.lower()].upper()
            else:
                plaintext += key[letter]
        else:
            plaintext += letter
            
    return plaintext

# example usagea
ciphertext = "Uif mjtu pg bqqmf isphfdujpo jt tjdifs xpsme"
key = frequency_analysis(ciphertext)
plaintext = decrypt(ciphertext, key)
print("Ciphertext:", ciphertext)
print("Plaintext:", plaintext)


implementation of a Hill cipher cryptanalysis algorithm in Python:

In [ ]:
import numpy as np
from collections import Counter

def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a % b)

def mod_inverse(a, m):
    a = a % m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    return 1

def hill_cryptanalysis(ciphertext, block_size, alphabet):
    ciphertext = ciphertext.upper()
    alphabet_size = len(alphabet)
    n = block_size
    
    # Determine expected letter frequencies for the plaintext language
    freqs = {'A': 0.0817, 'B': 0.0150, 'C': 0.0278, 'D': 0.0425, 'E': 0.1270, 
             'F': 0.0223, 'G': 0.0202, 'H': 0.0609, 'I': 0.0697, 'J': 0.0015, 
             'K': 0.0077, 'L': 0.0403, 'M': 0.0241, 'N': 0.0675, 'O': 0.0751, 
             'P': 0.0193, 'Q': 0.0010, 'R': 0.0599, 'S': 0.0633, 'T': 0.0906, 
             'U': 0.0276, 'V': 0.0098, 'W': 0.0236, 'X': 0.0015, 'Y': 0.0197, 'Z': 0.0007}
    
    # Determine the ciphertext blocks and their corresponding plaintext blocks
    ciphertext_blocks = [ciphertext[i:i+n] for i in range(0, len(ciphertext), n)]
    plaintext_blocks = []
    for block in ciphertext_blocks:
        block_freqs = Counter(block)
        block_freqs = {k: v/len(block) for k, v in block_freqs.items()}
        diff_freqs = {k: abs(v - freqs[k]) for k, v in block_freqs.items() if k in freqs}
        best_guess = ''.join([x[0] for x in sorted(diff_freqs.items(), key=lambda x: x[1])][:n])
        plaintext_blocks.append(best_guess)
    
    # Construct the equation system
    A = []
    B = []
    for i in range(len(plaintext_blocks)):
        a_row = []
        for j in range(n):
            a_row += [alphabet.index(plaintext_blocks[i][j])]
        A.append(a_row)
        b_row = []
        for j in range(n):
            b_row += [alphabet.index(ciphertext_blocks[i][j])]
        B.append(b_row)
    A = np.array(A)
    B = np.array(B)
    B = B.flatten()

    # Solve the equation system
    det = int(round(np.linalg.det(A))) % alphabet_size
    if gcd(det, alphabet_size) != 1:
        print('Error: key matrix is not invertible')
        return None
    inv_det = mod_inverse(det, alphabet_size)
    adj = inv_det * np.round(np.linalg.inv(A) * det)
    adj = adj % alphabet_size
    key = adj.dot(B) % alphabet_size
    
    # Decrypt the ciphertext using the found key
    key = key.reshape(n, n)
    plaintext


algorithm for cryptanalysis of transposition cipher in Python:

In [ ]:
def transposition_decrypt(ciphertext):
    # Determine possible key lengths
    key_lengths = []
    for i in range(1, len(ciphertext)):
        if len(ciphertext) % i == 0:
            key_lengths.append(i)

    # Iterate through each possible key length
    for key_length in key_lengths:
        # Divide ciphertext into segments of key_length
        segments = [ciphertext[i:i+key_length] for i in range(0, len(ciphertext), key_length)]
        
        # Rearrange segments based on alphabetical order of first letter
        rearranged_segments = [None] * len(segments)
        for i in range(key_length):
            column = [segment[i] for segment in segments]
            sorted_column = sorted(column)
            for j in range(len(segments)):
                if segments[j][i] == sorted_column[0]:
                    rearranged_segments[j] = segments[j]
                    sorted_column.pop(0)
                    break
        
        # Read rearranged segments as plaintext
        plaintext = ''
        for i in range(len(rearranged_segments[0])):
            for j in range(len(rearranged_segments)):
                plaintext += rearranged_segments[j][i]
        
        # Check if plaintext is valid (contains only letters and spaces)
        if all(c.isalpha() or c.isspace() for c in plaintext):
            return plaintext
    
    # If plaintext cannot be recovered, return empty string
    return ''


Cryptanalysis of Vigenère cipher using frequency analysis and the Kasiski test:

In [ ]:
import collections

def kasiski_test(ciphertext):
    # Find repeated sequences of three or more letters in the ciphertext
    repeated_sequences = collections.defaultdict(list)
    for i in range(len(ciphertext) - 2):
        seq = ciphertext[i:i+3]
        if seq in ciphertext[i+3:]:
            repeated_sequences[seq].append(i)

    # Calculate the distances between the occurrences of each repeated sequence
    distances = {}
    for seq, positions in repeated_sequences.items():
        distances[seq] = [positions[j+1] - positions[j] for j in range(len(positions)-1)]

    # Identify the factors of each distance and group them by distance
    factors = collections.defaultdict(list)
    for seq, dists in distances.items():
        for d in dists:
            for i in range(2, int(d**0.5)+1):
                if d % i == 0:
                    factors[d].extend([i, d//i])
            factors[d] = list(set(factors[d]))

    # Choose the most likely length based on the frequency of its factors
    likely_lengths = []
    for dist, factors_list in factors.items():
        for f in factors_list:
            if f > 2:
                length = dist // f
                if length > 2 and length not in likely_lengths:
                    likely_lengths.append(length)
    return likely_lengths


def decrypt_vigenere(ciphertext, keyword):
    plaintext = ''
    keyword_len = len(keyword)
    for i in range(len(ciphertext)):
        shift = ord(keyword[i % keyword_len]) - 65
        cipher_char = ciphertext[i]
        if cipher_char.isalpha():
            plaintext += chr((ord(cipher_char) - shift - 65) % 26 + 65)
        else:
            plaintext += cipher_char
    return plaintext


def cryptanalyze_vigenere(ciphertext):
    # Determine the length of the keyword using the Kasiski test
    likely_lengths = kasiski_test(ciphertext)
    print("Likely keyword lengths:", likely_lengths)

    # Divide the ciphertext into segments
    segments = []
    for i in range(min(likely_lengths), len(ciphertext)):
        if i in likely_lengths:
            segments.append([ciphertext[j] for j in range(0, len(ciphertext), i)])
    
    # Perform frequency analysis on each segment
    keyword = ''
    for segment in segments:
        # Calculate the frequency of each letter in the segment
        freq = collections.Counter(segment)
        most_common = freq.most_common(5)
        
        # Calculate the shift value for the most common letter (assuming it's 'E')
        shift = (ord(most_common[0][0]) - 69) % 26
        keyword += chr(shift + 65)
    
    # Determine the keyword
    print("Possible keywords:", keyword)

    # Decrypt the ciphertext using the keyword
    plaintext = decrypt_vigenere(ciphertext, keyword)
    return plaintext
